<a href="https://colab.research.google.com/github/sumanyurosha/tensorflow-specialization/blob/master/Hands-on%20ML/chapter13/IMDB_Movie_Reviews_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import tensorflow as tf
from tensorflow import keras
import os

# **Downloading the file structure**

In [30]:
from pathlib import Path

DOWNLOAD_ROOT = "http://ai.stanford.edu/~amaas/data/sentiment/"
FILENAME = "aclImdb_v1.tar.gz"
filepath = keras.utils.get_file(FILENAME, DOWNLOAD_ROOT + FILENAME, extract=True)


In [31]:
path = Path(filepath).parent / "aclImdb"
path

PosixPath('/root/.keras/datasets/aclImdb')

In [32]:
for name, subdirs, files in os.walk(path):
    indent = len(Path(name).parts) - len(path.parts)
    print("    " * indent + Path(name).parts[-1] + os.sep)
    for index, filename in enumerate(sorted(files)):
        if index == 3:
            print("    " * (indent + 1) + "...")
            break
        print("    " * (indent + 1) + filename)

aclImdb/
    README
    imdb.vocab
    imdbEr.txt
    test/
        labeledBow.feat
        urls_neg.txt
        urls_pos.txt
        pos/
            0_10.txt
            10000_7.txt
            10001_9.txt
            ...
        neg/
            0_2.txt
            10000_4.txt
            10001_1.txt
            ...
    train/
        labeledBow.feat
        unsupBow.feat
        urls_neg.txt
        ...
        pos/
            0_9.txt
            10000_8.txt
            10001_10.txt
            ...
        unsup/
            0_0.txt
            10000_0.txt
            10001_0.txt
            ...
        neg/
            0_3.txt
            10000_4.txt
            10001_4.txt
            ...


In [33]:
def list_review_paths(dirpath):
    return [path for path in dirpath.glob("*.txt")]

train_pos_reviews = list_review_paths(path / "train" / "pos")
train_neg_reviews = list_review_paths(path / "train" / "neg")
test_pos_reviews = list_review_paths(path / "test" / "pos")
test_neg_reviews = list_review_paths(path / "test" / "neg")

print(len(train_pos_reviews), len(train_neg_reviews), len(test_pos_reviews), len(test_neg_reviews))

12500 12500 12500 12500


### **Splitting the test set into Validation set and Test set**

In [35]:
import numpy as np

np.random.shuffle(test_pos_reviews)
np.random.shuffle(test_neg_reviews)

SPLIT_INDEX = 5000

test_pos = test_pos_reviews[:SPLIT_INDEX]
test_neg = test_neg_reviews[:SPLIT_INDEX]

valid_pos = test_pos_reviews[SPLIT_INDEX:]
valid_neg = test_neg_reviews[SPLIT_INDEX:]

print(len(test_pos), len(test_neg), len(valid_pos), len(valid_neg))

5000 5000 7500 7500


In [47]:
txt = tf.io.read_file(str(test_pos[0]))
txt

<tf.Tensor: shape=(), dtype=string, numpy=b'It is an interesting fact that metaphysics by Platon and Aristoteles, formal logic and abstract ontology form about those sciences that most people are not interested in. But then, around one thousand years after Aristoteles, the computer began to usurp the human thinking, and the humans who were refusing to reflect questions of being other than biological, physical and chemical ones, suddenly felt paralyzed because they could not cope with the consequences that this computers would bring "over night". R.W. Fassbinder\'s "Welt Am Draht", together with Tarkovsky\'s "Solaris" and Godard\'s "Alphaville", is probably the first movie who took the philosophical questions of emerging computer science as a basis of a story to be told in a movie. The confusing questions about identities and realities are cleverly built into different interwoven criminal stories which the audience really tries to follow because it is interested to solve the cases. Fass